In [1]:
import os
import io
import boto3
import gzip
import wget
from dotenv import load_dotenv, find_dotenv

This will take the gzip file and write the mbox to a new file. Still attempting to figure out how to do this without having to save locally and just write directly to bucket 

In [2]:
def gunzip(source_filepath, dest_filepath, block_size=65536):
    with gzip.open(source_filepath, 'rb') as s_file, \
            open(dest_filepath, 'wb') as d_file:
        while True:
            block = s_file.read(block_size)
            if not block:
                break
            else:
                d_file.write(block)

This retrieves the gzip from the fedora archieves for the time interval specified. From there it is unzipped and saved locally

In [3]:
f_list = input("Are you looking at dev or user? ")

if f_list == 'dev':
    s = "https://lists.fedoraproject.org/archives/list/devel@lists.fedoraproject.org/export/devel@lists.fedoraproject.org.mbox.gz?start="
    name = "../data//devel@lists.fedoraproject.org.mbox.gz"
elif f_list == 'user':
    s = "https://lists.fedoraproject.org/archives/list/users@lists.fedoraproject.org/export/users@lists.fedoraproject.org.mbox.gz?start="
    name = "../data//users@lists.fedoraproject.org.mbox.gz"

start_year = input("What year do you want to start with? example 2016: " )
start_month = input("What month do you want to start with? example 01: " )

finish_year = input("What year do you want to end with? example 2016: " )
finish_month = input("What month do you want to end with? example 01: " )
finish_month = str((int(finish_month)%12)+1)

s = f"{s}{start_year}-{start_month}-01&end={finish_year}-{finish_month}-01"
print(s)

wget.download(s, out="../data/")

gunzip(name,"/opt/app-root/src/trying.mbox" )

            

Are you looking at dev or user? dev
What year do you want to start with? example 2016: 2017
What month do you want to start with? example 01: 10
What year do you want to end with? example 2016: 2018
What month do you want to end with? example 01: 10
https://lists.fedoraproject.org/archives/list/devel@lists.fedoraproject.org/export/devel@lists.fedoraproject.org.mbox.gz?start=2017-10-01&end=2018-11-01


In [6]:
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)
s3_secret_key = os.environ['AWS_SECRET_ACCESS_KEY']
s3_bucket_name = os.environ['JUPYTERHUB_USER']
s3_endpoint_url = os.environ['S3_ENDPOINT_URL']
s3_access_key = os.environ['AWS_ACCESS_KEY_ID']
s3bucket = os.environ['BUCKET']

print(s3_endpoint_url)
print(s3_bucket_name)
s3 = boto3.client('s3','us-east-1', endpoint_url= s3_endpoint_url,
                       aws_access_key_id = s3_access_key,
                       aws_secret_access_key = s3_secret_key)

https://s3.upshift.redhat.com
cdolfi


In [10]:

s3_location = "cdolfi/mboxes/" + start_year + "_" + start_month + "-" + finish_year + "_"+  finish_month + ".mbox"
s3.upload_file(Filename="/opt/app-root/src/trying.mbox",Bucket=s3bucket, Key=s3_location)

In [5]:
import mailbox
msgs = mailbox.mbox("/opt/app-root/src/trying.mbox")

In [7]:
s3_location = "cdolfi/gzip/" + start_year + "_" + start_month + "-" + finish_year + "_"+  finish_month + ".gzip"
s3.upload_file(Filename=name,Bucket=s3bucket, Key=s3_location)